# Imports

to do: 
- JF: add MATLAB changes (raw waveforms, RPV, spikeGLX meta, spike clusters, amplitude outliers)
- what are the errors / warnings in the main function?
- JF: generate output plots 
- how to load output, check and modify param, then re-generate + save 
- double check no hard-coding 
- double check all python names are copies of MATLAB with snake_case
- check comments & formatting 

In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add bombcell to Python path if NOT installed with pip
demo_dir = Path(os.getcwd())
pyBombCell_dir = demo_dir.parent
sys.path.append(str(pyBombCell_dir))

In [2]:
%load_ext autoreload
%autoreload 2

import bombcell as bc

/home/julie/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/julie/.local/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Define data paths

By default: path to BombCell's toy dataset

In [3]:
ks_dir = demo_dir / "toy_data"  # Replace with your kilosort directory
raw_dir = (
    None  # Leave 'None' if no raw data; eventually replace with path to your raw data
)
save_path = "~/Downloads/bombcell_plots"  # ~ is home directory, / work on Windows

# If a raw data directory with a meta folder is not given,
# please input the gain manually
gain_to_uV = np.nan

In [14]:
# ephys_raw_data and gain_to_uv will be None if no raw_dir given
ephys_raw_data, meta_path, gain_to_uV = bc.manage_if_raw_data(raw_dir)

param = bc.get_default_parameters(ks_dir, raw_dir=raw_dir, ephys_meta_dir=meta_path)
param["compute_distance_metrics"] = 1
param["compute_drift"] = 1
param["compute_time_chunks"] = 1;

Run bombcell, get unit types and save results 
To save results as a parquet either PyArrow or FastParquet needs to be installed

In [ ]:
(
    quality_metrics,
    param,
    unique_templates,
    raw_waveforms_full,
    raw_waveforms_peak_channel,
    unit_type,
    unit_type_string,
) = bc.run_bombcell(
    ks_dir, raw_dir, save_path, gain_to_uV, param
)  # JF: only input could be param here

Computing BombCell quality metrics:   0%|          | 0/15 units/home/julie/.local/lib/python3.8/site-packages/scipy/optimize/minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/julie/Dropbox/MATLAB/onPaths/PyBombcell/pyBombCell/bombcell/quality_metrics.py:654: RuntimeWarning: invalid value encountered in divide
  spike_depth_in_channels = np.sum(pc_channel_pos_weights[np.newaxis, :] * pc_features_pc1 ** 2, axis = 1) / np.nansum(pc_features_pc1 ** 2, axis = 1)
Computing BombCell quality metrics:  33%|███▎      | 5/15 units/home/julie/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/julie/Dropbox/MATLAB/onPaths/PyBombcell/pyBombCell/bombcell/quality_metrics.py:334: RuntimeWarning: divide by zero encountered in scalar divide
  p_missing = ((surrogate_area - np.sum(s

(optional) look at a table which shows why each unit fails

In [ ]:
qm_table = bc.make_qm_table(
    quality_metrics, param, unique_templates, unit_type_string
)  # JF: does not include distance and drift metrics?
qm_table

,Original ID,NaN result,Peaks,Troughs,Waveform Min Length,Waveform Max Length,Baseline,Spatial Decay,Min Spikes,Missing Spikes,RPVs,Presence Ratio,Somatic,Good Unit
0,0,False,False,False,False,False,False,False,False,True,True,False,True,MUA
1,1,False,False,True,False,False,True,True,False,False,True,False,False,NOISE
2,2,False,False,False,False,False,False,False,False,False,True,False,True,MUA
3,3,False,False,False,False,False,False,False,True,False,False,False,True,MUA
4,4,False,False,False,False,False,False,False,False,False,True,False,True,MUA
5,5,False,False,False,False,False,False,False,True,True,False,False,True,MUA
6,6,False,False,False,False,False,False,False,False,False,False,False,True,MUA
7,7,False,False,False,False,False,False,False,True,False,False,True,True,MUA
8,8,False,False,False,False,False,False,False,True,False,True,False,True,MUA
9,9,False,False,False,False,False,False,False,True,False,False,True,True,MUA
